In [1]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [2]:
# System imports
import os
import sys
import yaml

# External imports
import matplotlib.pyplot as plt
import scipy as sp
from sklearn.decomposition import PCA
from sklearn.metrics import auc
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import nersc_tensorboard_helper

sys.path.append('..')
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from LightningModules.GNN.Models.ecgnn import ECGNN

In [4]:
with open("example_gnn.yaml") as f:
        hparams = yaml.load(f, Loader=yaml.FullLoader)

In [5]:
model = ECGNN(hparams)
logger = TensorBoardLogger("tb_ecgnn_logs",name = "ecgnn")
trainer = Trainer(gpus=1, max_epochs=10,logger = logger)
trainer.fit(model)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name             | Type        | Params
-------------------------------------------------
0 | input_network    | Sequential  | 384   
1 | node_network     | NodeNetwork | 17.2 K
2 | edgepool_network | EdgePooling | 66.2 K
-------------------------------------------------
83.8 K    Trainable params
0         Non-trainable params
83.8 K    Total params


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/220 [00:00<?, ?it/s]                      

/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:  91%|█████████ | 200/220 [07:20<00:44,  2.20s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 1:  91%|█████████ | 200/220 [07:21<00:44,  2.21s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 2:  91%|█████████ | 200/220 [07:20<00:44,  2.20s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 3:  91%|█████████ | 200/220 [07:21<00:44,  2.21s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 4:  91%|█████████ | 200/220 [07:25<00:44,  2.23s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 5:  91%|█████████ | 200/220 [07:20<00:44,  2.20s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 6:  91%|█████████ | 200/220 [07:21<00:44,  2.21s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 7:  91%|█████████ | 200/220 [07:23<00:44,  2.22s/it, loss=0.918, v_num=1859877]
Validating: 0it [00:00, ?it/s]
Epoch 8:  91%|█████████ | 200/220 [07:19<00:43,  2.20s/it, loss=

1

In [6]:
test_results = trainer.test(ckpt_path=None)
print(test_results)

Set SLURM handle signals.


Testing:   0%|          | 0/20 [00:00<?, ?it/s]

/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 100%|██████████| 20/20 [00:47<00:00,  2.37s/it]
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'current_lr': 0.00029999998514540493,
 'eff': 0.5479669631512071,
 'loss': 0.9258258938789368,
 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32),
 'val_loss': 0.9154987931251526}
--------------------------------------------------------------------------------
[{'loss': 0.9258258938789368, 'eff': 0.5479669631512071, 'truth': array([0., 1., 0., ..., 0., 1., 1.], dtype=float32), 'val_loss': 0.9154987931251526, 'current_lr': 0.00029999998514540493}]


In [4]:
os.environ['TENSORBOARD_BINARY'] = "/global/homes/j/jferguso/.conda/envs/exatrkx-tracking/bin/tensorboard"
%tensorboard --logdir tb_ecgnn_logs/ --port 0
nersc_tensorboard_helper.tb_address()


/bin/bash: line 0: kill: (67561) - No such process
